In [1]:
%run helper/setup_notebook.ipynb import display_table

Successfully connected to leetcode50 database.


In [2]:
tables = ['Movies', 'Users', 'MovieRating']

for table in tables:
    display_table(table)

+----------+----------+
| movie_id |  title   |
+----------+----------+
|    1     | Avengers |
|    2     | Frozen 2 |
|    3     |  Joker   |
+----------+----------+
+---------+--------+
| user_id |  name  |
+---------+--------+
|    1    | Daniel |
|    2    | Monica |
|    3    | Maria  |
|    4    | James  |
+---------+--------+
+----------+---------+--------+------------+
| movie_id | user_id | rating | created_at |
+----------+---------+--------+------------+
|    1     |    1    |   3    | 2020-01-12 |
|    1     |    2    |   4    | 2020-02-11 |
|    1     |    3    |   2    | 2020-02-12 |
|    1     |    4    |   1    | 2020-01-01 |
|    2     |    1    |   5    | 2020-02-17 |
|    2     |    2    |   2    | 2020-02-01 |
|    2     |    3    |   2    | 2020-03-01 |
|    3     |    1    |   3    | 2020-02-22 |
|    3     |    2    |   4    | 2020-02-25 |
+----------+---------+--------+------------+


### Write an SQL query to:
* #### Find the name of the user who has rated the greatest number of movies. In case of a tie, return the lexicographically smaller user name.
* #### Find the movie name with the highest average rating in February 2020. In case of a tie, return the lexicographically smaller movie name.

### The query result format is in the following example.*

```
+--------------+
| results      |
+--------------+
| Daniel       |
| Frozen 2     |
+--------------+
Explanation: 

Daniel and Monica have rated 3 movies ("Avengers", "Frozen 2" and "Joker") but Daniel is smaller lexicographically.

Frozen 2 and Joker have a rating average of 3.5 in February but Frozen 2 is smaller lexicographically.
```

In [3]:
%%sql 

SELECT 
    user_id,
    COUNT(user_id) AS vote_count
FROM MovieRating
GROUP BY user_id;


user_id,vote_count
1,3
2,3
3,2
4,1


In [4]:
%%sql 

SELECT 
    user_id
FROM MovieRating
GROUP BY user_id
HAVING COUNT(user_id) = (
    SELECT COUNT(user_id) 
    FROM MovieRating 
    GROUP BY user_id 
    LIMIT 1)

user_id
1
2


In [5]:
%%sql 

SELECT 
    name
FROM MovieRating
JOIN Users ON Users.user_id = MovieRating.user_id
GROUP BY name
HAVING COUNT(Users.user_id) = (
    SELECT COUNT(user_id) 
    FROM MovieRating
    GROUP BY user_id 
    LIMIT 1)

name
Daniel
Monica


In [6]:
%%sql 

SELECT 
    name
FROM MovieRating
JOIN Users ON Users.user_id = MovieRating.user_id
GROUP BY name
HAVING COUNT(Users.user_id) = (
    SELECT COUNT(user_id) 
    FROM MovieRating
    GROUP BY user_id 
    LIMIT 1)
ORDER BY name 
LIMIT 1

name
Daniel


### Second query

In [7]:
%%sql 

SELECT
    movie_id,
    AVG(rating)
FROM MovieRating
GROUP BY movie_id

movie_id,AVG(rating)
1,2.5000
2,3.0000
3,3.5000


### Get the month column

In [8]:
%%sql 

SELECT SUBSTR(created_at, 1, 7) AS month
FROM MovieRating

month
2020-01
2020-02
2020-02
2020-01
2020-02
2020-02
2020-03
2020-02
2020-02


In [9]:
%%sql 

SELECT
    Movies.movie_id,
    title,
    AVG(rating) AS average_rating
FROM MovieRating 
JOIN Movies ON Movies.movie_id = MovieRating.movie_id
WHERE SUBSTRING(created_at, 6, 2) = 2
GROUP BY movie_id, title

movie_id,title,average_rating
1,Avengers,3.0000
2,Frozen 2,3.5000
3,Joker,3.5000


In [10]:
%%sql 

SELECT AVG(rating) AS average_rating
FROM MovieRating
GROUP BY movie_id
ORDER BY average_rating DESC

average_rating
3.5000
3.0000
2.5000


In [11]:
%%sql 

SELECT AVG(rating) AS average_rating
FROM MovieRating
GROUP BY movie_id
ORDER BY average_rating DESC
LIMIT 1

average_rating
3.5000


In [12]:
%%sql 

SELECT
    Movies.movie_id,
    AVG(rating) AS average_rating
FROM MovieRating 
JOIN Movies ON Movies.movie_id = MovieRating.movie_id 
WHERE SUBSTR(created_at, 1, 7) = "2020-02"
GROUP BY movie_id
HAVING average_rating = (
    SELECT AVG(rating) AS average_rating
    FROM MovieRating
    GROUP BY movie_id
    ORDER BY average_rating DESC
    LIMIT 1)

movie_id,average_rating
2,3.5000
3,3.5000


In [13]:
%%sql 

SELECT
    title
FROM MovieRating 
JOIN Movies ON Movies.movie_id = MovieRating.movie_id 
WHERE SUBSTR(created_at, 1, 7) = "2020-02"
GROUP BY title
HAVING AVG(rating) = (
    SELECT AVG(rating) AS average_rating
    FROM MovieRating
    GROUP BY movie_id
    ORDER BY average_rating DESC
    LIMIT 1)

title
Frozen 2
Joker


In [14]:
%%sql 

SELECT
    title
FROM MovieRating 
JOIN Movies ON Movies.movie_id = MovieRating.movie_id 
WHERE SUBSTR(created_at, 1, 7) = "2020-02"
GROUP BY title
HAVING AVG(rating) = (
    SELECT AVG(rating) AS average_rating
    FROM MovieRating
    GROUP BY movie_id
    ORDER BY average_rating DESC
    LIMIT 1)
ORDER BY title
LIMIT 1

title
Frozen 2


### Combine both results

In [15]:
%%sql 

(SELECT 
    name AS results
FROM MovieRating
JOIN Users ON Users.user_id = MovieRating.user_id
GROUP BY name
HAVING COUNT(Users.user_id) = (
    SELECT COUNT(user_id) 
    FROM MovieRating
    GROUP BY user_id 
    LIMIT 1)
ORDER BY name 
LIMIT 1)

UNION 

(SELECT
    title AS results
FROM MovieRating 
JOIN Movies ON Movies.movie_id = MovieRating.movie_id 
WHERE SUBSTR(created_at, 1, 7) = "2020-02"
GROUP BY title
HAVING AVG(rating) = (
    SELECT AVG(rating) AS average_rating
    FROM MovieRating
    GROUP BY movie_id
    ORDER BY average_rating DESC
    LIMIT 1)
ORDER BY title
LIMIT 1)

results
Daniel
Frozen 2


# Using Pandas

In [16]:
movies_query = %sql SELECT * FROM movies # type: ignore
users_query = %sql SELECT * FROM users # type: ignore
movie_rating_query = %sql SELECT * FROM MovieRating # type: ignore

movies_df = movies_query.DataFrame()
users_df = users_query.DataFrame()
movie_rating_df = movie_rating_query.DataFrame()

display(movies_df, users_df, movie_rating_df)

,movie_id,title
0,1,Avengers
1,2,Frozen 2
2,3,Joker


,user_id,name
0,1,Daniel
1,2,Monica
2,3,Maria
3,4,James


,movie_id,user_id,rating,created_at
0,1,1,3,2020-01-12
1,1,2,4,2020-02-11
2,1,3,2,2020-02-12
3,1,4,1,2020-01-01
4,2,1,5,2020-02-17
5,2,2,2,2020-02-01
6,2,3,2,2020-03-01
7,3,1,3,2020-02-22
8,3,2,4,2020-02-25


In [17]:
merged_df = movie_rating_df.merge(users_df, on='user_id')
merged_df

,movie_id,user_id,rating,created_at,name
0,1,1,3,2020-01-12,Daniel
1,2,1,5,2020-02-17,Daniel
2,3,1,3,2020-02-22,Daniel
3,1,2,4,2020-02-11,Monica
4,2,2,2,2020-02-01,Monica
5,3,2,4,2020-02-25,Monica
6,1,3,2,2020-02-12,Maria
7,2,3,2,2020-03-01,Maria
8,1,4,1,2020-01-01,James


In [18]:
merged_df.groupby('name').agg(results=('name', 'count'))

,results
name,
Daniel,3
James,1
Maria,2
Monica,3


In [19]:
merged_df.groupby('name').agg(results=('name', 'count')).sort_values('results', ascending=False)

,results
name,
Daniel,3
Monica,3
Maria,2
James,1


In [20]:
merged_df.groupby('name') \
    .agg(results=('name', 'count')) \
    .sort_values(['results','name'], ascending=[False, True]) 

,results
name,
Daniel,3
Monica,3
Maria,2
James,1


In [21]:
final_df_1 = merged_df.groupby('name') \
    .agg(results=('name', 'count')) \
    .sort_values(['results','name'], ascending=[False, True]) \
    .reset_index() \
    .head(1)[['name']]
    
final_df_1

,name
0,Daniel


In [22]:
final_df_1.rename(columns={'name': 'results'}, inplace=True)
final_df_1

,results
0,Daniel


In [23]:
merged_df = movie_rating_df.merge(movies_df, on='movie_id')
merged_df

,movie_id,user_id,rating,created_at,title
0,1,1,3,2020-01-12,Avengers
1,1,2,4,2020-02-11,Avengers
2,1,3,2,2020-02-12,Avengers
3,1,4,1,2020-01-01,Avengers
4,2,1,5,2020-02-17,Frozen 2
5,2,2,2,2020-02-01,Frozen 2
6,2,3,2,2020-03-01,Frozen 2
7,3,1,3,2020-02-22,Joker
8,3,2,4,2020-02-25,Joker


In [24]:
pd.DatetimeIndex(merged_df['created_at']).month

Index([1, 2, 2, 1, 2, 2, 3, 2, 2], dtype='int32', name='created_at')

In [25]:
pd.DatetimeIndex(merged_df['created_at']).year

Index([2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020], dtype='int32', name='created_at')

In [26]:
merged_df[(pd.DatetimeIndex(merged_df['created_at']).month == 2)]

,movie_id,user_id,rating,created_at,title
1,1,2,4,2020-02-11,Avengers
2,1,3,2,2020-02-12,Avengers
4,2,1,5,2020-02-17,Frozen 2
5,2,2,2,2020-02-01,Frozen 2
7,3,1,3,2020-02-22,Joker
8,3,2,4,2020-02-25,Joker


In [27]:
fitlered_df = merged_df[(pd.DatetimeIndex(merged_df['created_at']).month == 2) & 
                        (pd.DatetimeIndex(merged_df['created_at']).year == 2020)]

fitlered_df

,movie_id,user_id,rating,created_at,title
1,1,2,4,2020-02-11,Avengers
2,1,3,2,2020-02-12,Avengers
4,2,1,5,2020-02-17,Frozen 2
5,2,2,2,2020-02-01,Frozen 2
7,3,1,3,2020-02-22,Joker
8,3,2,4,2020-02-25,Joker


In [28]:
fitlered_df.groupby('title').agg(results=('rating', 'mean'))

,results
title,
Avengers,3.0
Frozen 2,3.5
Joker,3.5


In [29]:
fitlered_df.groupby('title') \
    .agg(results=('rating', 'mean')) \
    .sort_values(['results', 'title'], ascending=[False, True])

,results
title,
Frozen 2,3.5
Joker,3.5
Avengers,3.0


In [30]:
final_df_2 = fitlered_df.groupby('title') \
    .agg(results=('rating', 'mean')) \
    .sort_values(['results', 'title'], ascending=[False, True]) \
    .reset_index() \
    .head(1)[['title']]
    
final_df_2

,title
0,Frozen 2


In [31]:
final_df_2.rename(columns={'title': 'results'}, inplace=True)
final_df_2

,results
0,Frozen 2


In [32]:
pd.concat([final_df_1, final_df_2], ignore_index=True)

,results
0,Daniel
1,Frozen 2
